In [1]:
%%html

<style>    
    @import url("css/custom_styles.css")
</style>

# Rapport final

## Jean-Francois Gagnon, Michele De La Sablonniere et Frederic Champagne

Introduction(Explication):

1. On s’intéresse à développer un modèle de prédiction de classes. Identifier d’abord les classes disponibles dans le dataset que vous avez choisi.

## Description

Dans cette base de données, il y a 39 differentes classes d'image de feuilles. La base de donnée contient 61,486 images.

Les classes sont: 

<br>
<div>
<div class="columns_2">
    
1. Apple_scab
2. Apple_black_rot
3. Apple_cedar_apple_rust
4. Apple_healthy
5. Background_without_leaves
6. Blueberry_healthy
7. Cherry_powdery_mildew
8. Cherry_healthy
9. Corn_gray_leaf_spot
10. Corn_common_rust
11. Corn_northern_leaf_blight
12. Corn_healthy
13. Grape_black_rot
14. Grape_black_measles
15. Grape_leaf_blight
16. Grape_healthy
17. Orange_haunglongbing
18. Peach_bacterial_spot
19. Peach_healthy
20. Pepper_bacterial_spot

</div>

<div class="columns_2">
    
21. Pepper_healthy
22. Potato_early_blight
23. Potato_healthy
24. Potato_late_blight
25. Raspberry_healthy
26. Soybean_healthy
27. Squash_powdery_mildew
28. Strawberry_healthy
29. Strawberry_leaf_scorch
30. Tomato_bacterial_spot
31. Tomato_early_blight
32. Tomato_healthy
33. Tomato_late_blight
34. Tomato_leaf_mold
35. Tomato_septoria_leaf_spot
36. Tomato_spider_mites_two-spotted_spider_mite
37. Tomato_target_spot
38. Tomato_mosaic_virus
39. Tomato_yellow_leaf_curl_virus
    
</div>
</div>

2. Procéder à une exploration du dataset et identifier des valeurs manquantes, inconnues, etc. Procéder au remplacement de ces valeurs s’il y’a lieu.

3. Développer la matrice de corrélation. Interpréter vos résultats.

4. Procéder à la séparation train/test (80-20) de votre dataset original (seed 44).

5. Utilisation et comparaison de différents modèles.

6. Calculer l’exactitude (Accuracy) sur la base des valeurs test.

7. Sauvegarde du meilleur modèle pour utilisation ultérieure.

8. Utilisation du modèle sauvegardé pour la prédiction.